In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from glob import glob
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, MaxPool2D, Add, Dropout, Concatenate, Conv2DTranspose, Dense, Reshape, Flatten, Softmax, Lambda, UpSampling2D, AveragePooling2D, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D, SeparableConv2D,Multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import pandas as pd

2024-06-16 13:22:04.305422: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 13:22:04.305485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 13:22:04.307100: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from sklearn.datasets import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras import backend
from math import pi
from math import cos
from math import floor

In [3]:
class ISICDataLoader:
    def __init__(self, data_dir, image_size=(256, 256), batch_size=8, mode='train'):
        self.data_dir = data_dir
        self.image_size = image_size
        self.batch_size = batch_size
        self.mode = mode

        # Load CSV files based on the mode
        if self.mode == 'train':
            csv_filename = "/kaggle/input/isic-2017-segmentation/train_ph2.csv"
        elif self.mode == 'val':
            csv_filename = "/kaggle/input/isic-2017-segmentation/val_ph2.csv"
        else:
            csv_filename = "/kaggle/input/isic-2017-segmentation/test_ph2.csv"

        self.csv_path = os.path.join(self.data_dir, csv_filename)
        self.df = pd.read_csv(self.csv_path)

        # Define image and mask paths
        self.image_path = os.path.join(self.data_dir, "Images")
        self.mask_path = os.path.join(self.data_dir, "Masks")

        # Initialize the current batch index to 0
        self.current_batch_index = 0

    def __len__(self):
        return int(np.ceil(len(self.df) / float(self.batch_size)))

    def __iter__(self):
        while self.current_batch_index < len(self.df):
            batch_images = []
            batch_masks = []
            for i in range(self.current_batch_index, min(self.current_batch_index + self.batch_size, len(self.df))):
                image_name = self.df.iloc[i]['Image_Name']
                mask_name = image_name[:-4]+'_lesion'+'.bmp'

                image = load_img(os.path.join(self.image_path, image_name), target_size=self.image_size)
                mask = load_img(os.path.join(self.mask_path, mask_name), target_size=self.image_size, color_mode='grayscale')

                image_arr = img_to_array(image) / 255.0
                mask_arr = img_to_array(mask) / 255.0

                batch_images.append(image_arr)
                batch_masks.append(mask_arr)

            batch_images = np.array(batch_images)
            batch_masks = np.array(batch_masks)

            # Update the current batch index for the next iteration
            self.current_batch_index += self.batch_size

            yield batch_images, batch_masks
            
        # Reset the current batch index at the end of one epoch
        self.current_batch_index = 0

In [4]:
train_data_loader = ISICDataLoader(data_dir='/kaggle/input/isic-2017-segmentation/Arranged_PH2_dataset/Arranged_PH2_dataset', image_size=(512, 512), batch_size=4, mode='train')
val_data_loader = ISICDataLoader(data_dir='/kaggle/input/isic-2017-segmentation/Arranged_PH2_dataset/Arranged_PH2_dataset', image_size=(512, 512), batch_size=4, mode='val')
test_data_loader = ISICDataLoader(data_dir='/kaggle/input/isic-2017-segmentation/Arranged_PH2_dataset/Arranged_PH2_dataset', image_size=(512, 512), batch_size=4, mode='test')

train_dataset = tf.data.Dataset.from_generator(lambda: train_data_loader, output_signature=(tf.TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32), tf.TensorSpec(shape=(None, 512, 512, 1), dtype=tf.float32)))
val_dataset = tf.data.Dataset.from_generator(lambda: val_data_loader, output_signature=(tf.TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32), tf.TensorSpec(shape=(None, 512, 512, 1), dtype=tf.float32)))
test_dataset = tf.data.Dataset.from_generator(lambda: test_data_loader, output_signature=(tf.TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32), tf.TensorSpec(shape=(None, 512, 512, 1), dtype=tf.float32)))

In [5]:
def dice_score(y_true, y_pred):
    smooth = K.epsilon()
    y_true_flat = K.flatten(K.cast(y_true, 'float32'))
    y_pred_flat = K.flatten(y_pred)
    intersection = K.sum(y_true_flat * y_pred_flat)
    score = (2. * intersection + smooth) / (K.sum(y_true_flat) + K.sum(y_pred_flat) + smooth)
    return score

def iou(y_true, y_pred):
    smooth = K.epsilon()
    y_true_flat = K.flatten(K.cast(y_true, 'float32'))
    y_pred_flat = K.flatten(y_pred)
    intersection = K.sum(y_true_flat * y_pred_flat)
    union = K.sum(y_true_flat) + K.sum(y_pred_flat) - intersection + smooth
    iou = (intersection + smooth) / union
    return iou

def recall(y_true, y_pred):
    smooth = K.epsilon()
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_true_flat = K.flatten(K.cast(y_true, 'float32'))
    y_pred_flat = K.flatten(y_pred_pos)
    tp = K.sum(y_true_flat * y_pred_flat)
    fn = K.sum(y_true_flat * (1 - y_pred_flat))
    recall = (tp + smooth) / (tp + fn + smooth)
    return recall

def precision(y_true, y_pred):
    smooth = K.epsilon()
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_true_flat = K.flatten(K.cast(y_true, 'float32'))
    y_pred_flat = K.flatten(y_pred_pos)
    tp = K.sum(y_true_flat * y_pred_flat)
    fp = K.sum((1 - y_true_flat) * y_pred_flat)
    precision = (tp + smooth) / (tp + fp + smooth)
    return precision

In [6]:
def dice_loss(y_true, y_pred):
    loss = 1 - dice_score(y_true, y_pred)
    return loss

def iou_loss(y_true, y_pred):
    loss = 1 - iou(y_true, y_pred)
    return loss
    
def focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    epsilon = tf.keras.backend.epsilon()
    y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
    y_true = tf.cast(y_true, tf.float32)
    pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
    focal_weight = alpha * tf.pow(1 - pt, gamma)
    loss = tf.reduce_mean(-focal_weight * tf.math.log(pt))
    return loss

def bce_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred))
    return loss

def combined_loss(y_true, y_pred):
    loss = dice_loss(y_true, y_pred) + bce_loss(y_true, y_pred)
    return loss

In [7]:
class SnapshotEnsemble(Callback):
    def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
        super(SnapshotEnsemble, self).__init__()
        self.epochs = n_epochs
        self.cycles = n_cycles
        self.lr_max = lrate_max
        self.lrates = list()

    def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
        epochs_per_cycle = floor(n_epochs / n_cycles)
        cos_inner = (pi * (epoch % epochs_per_cycle)) / epochs_per_cycle
        return lrate_max / 2 * (cos(cos_inner) + 1)

    def on_epoch_begin(self, epoch, logs=None):
        lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
        self.model.optimizer.learning_rate = lr
        self.lrates.append(lr)
    
    def on_epoch_end(self, epoch, logs={}):
        # check if we can save model
        epochs_per_cycle = (self.epochs / self.cycles)
        if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
        # save model to file
            filename = "snapshot_model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
            self.model.save(filename)
            print('>saved snapshot %s, epoch %d' % (filename, epoch))

In [8]:
def conv_block(x, num_filters, kernel_size, padding="same", use_bias=False, act=True):
    x = Conv2D(num_filters, kernel_size, padding=padding, use_bias=use_bias)(x)
    x = BatchNormalization()(x)
    if act:
        x = Activation("relu")(x)
    return x

def region_aware_attention(x):
    num_filters = x.shape[-1]
    max_pool = GlobalMaxPooling2D()(x)
    avg_pool = GlobalAveragePooling2D()(x)
    
    concat = Concatenate()([max_pool, avg_pool])
    dense = Dense(num_filters, activation='sigmoid')(concat)
    attention = Reshape((1, 1, num_filters))(dense)
    attention = Multiply()([x, attention])
    return attention

def encoder_block(x, num_filters):
    x = conv_block(x, num_filters, 3)
    x = conv_block(x, num_filters, 3)
    s = x
    p = MaxPooling2D((2, 2))(x)
    return s, p

def decoder_block(x, skip, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(x)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters, 3)
    x = conv_block(x, num_filters, 3)
    return x

def build_ra_net(shape):
    inputs = Input(shape)

    s1, p1 = encoder_block(inputs, 32)
    s2, p2 = encoder_block(p1, 64)
    s3, p3 = encoder_block(p2, 128)
    s4, p4 = encoder_block(p3, 256)

    bridge = conv_block(p4, 512, 3)
    bridge = region_aware_attention(bridge)

    d1 = decoder_block(bridge, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="RA-Net")
    return model


optimizer = Adam(learning_rate=0.0001)  # Moved optimizer initialization here
model = build_ra_net((512, 512, 3))
model.compile(loss=combined_loss, metrics=["accuracy", dice_score, recall, precision, iou], optimizer=optimizer)
model.summary()

Model: "RA-Net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 512, 512,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512, 512,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 512, 512,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 512, 512,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 512,  │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 512, 512,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 256, 256,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 256, 256,  │     18,432 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 256,  │     36,864 │ activation_2[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 128, 128,  │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │     73,728 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_4[0][0]  

 Total params: 5,932,385 (22.63 MB)

 Trainable params: 5,927,521 (22.61 MB)

 Non-trainable params: 4,864 (19.00 KB)

In [9]:
model.load_weights("/kaggle/input/ph2-ra/ph2-RA/snapshot_model_1.h5")
prediction1 = model.predict(test_dataset)

10/10 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [10]:
print(prediction1.shape)

(40, 512, 512, 1)


In [11]:
model.load_weights("/kaggle/input/ph2-ra/ph2-RA/snapshot_model_2.h5")
prediction2 = model.predict(test_dataset)

10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step


In [12]:
print(prediction2.shape)

(40, 512, 512, 1)


In [13]:
model.load_weights("/kaggle/input/ph2-ra/ph2-RA/snapshot_model_3.h5")
prediction3 = model.predict(test_dataset)

10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step


In [14]:
print(prediction3.shape)

(40, 512, 512, 1)


In [15]:
model.load_weights("/kaggle/input/ph2-ra/ph2-RA/snapshot_model_4.h5")
prediction4 = model.predict(test_dataset)

10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step


In [16]:
print(prediction4.shape)

(40, 512, 512, 1)


In [17]:
model.load_weights("/kaggle/input/ph2-ra/ph2-RA/snapshot_model_5.h5")
prediction5 = model.predict(test_dataset)

10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step


In [18]:
print(prediction5.shape)

(40, 512, 512, 1)


In [19]:
threshold = 0.5     
threshold1 = 0.35
threshold2 = 0.65
thre_interval=(threshold2-threshold1)
bias = 0.001
thre_bnd = .455

In [20]:
pred_mask1 = (prediction1 >= threshold).astype(int)                       
pred_mask2 = (prediction2 >= threshold).astype(int)    
pred_mask3 = (prediction3 >= threshold).astype(int)   
pred_mask4 = (prediction4 >= threshold).astype(int)
pred_mask5 = (prediction5 >= threshold).astype(int) 

In [21]:
y_test = []

for images, masks in test_data_loader:
    y_test.extend(masks)

y_test = np.array(y_test)

In [22]:
print(y_test.shape)

(40, 512, 512, 1)


In [23]:
predictions = np.array([prediction1, prediction2, prediction3, prediction4, prediction5])

In [24]:
ensemble_mean_prediction = np.mean(predictions, axis=0)                                 
ensemble_std_prediction = np.std(predictions, axis=0)

In [25]:
ensemble_prediction = np.zeros(ensemble_mean_prediction.shape)

In [26]:
ensemble_mean_prediction=np.where(ensemble_mean_prediction>=threshold2,1,(np.where(((ensemble_mean_prediction>=threshold1) & (ensemble_mean_prediction<threshold2) & (((ensemble_mean_prediction-threshold1)/thre_interval)  +   bias>=thre_bnd)),1,0)))

In [27]:
ensemble_mask=ensemble_mean_prediction

In [28]:
def dice_score1(y_true, y_pred):       
    smooth = np.finfo(np.float32).eps
    y_true_flat = y_true.flatten().astype('float32')       
    y_pred_flat = y_pred.flatten()    
    intersection = np.sum(y_true_flat * y_pred_flat)    
    score = (2. * intersection + smooth) / (np.sum(y_true_flat) + np.sum(y_pred_flat) + smooth)
    return score

In [30]:
dice1 = dice_score1(y_test, pred_mask1.squeeze())                                    
dice2 = dice_score1(y_test, pred_mask2.squeeze())         
dice3 = dice_score1(y_test, pred_mask3.squeeze())          
dice4 = dice_score1(y_test, pred_mask4.squeeze())          
dice5 = dice_score1(y_test, pred_mask5.squeeze())  

In [31]:
print("Dice Score for model1:", dice1)                                    
print("Dice Score for model2:", dice2)               
print("Dice Score for model3:", dice3)      
print("Dice Score for model4:", dice4)      
print("Dice Score for model5:", dice5) 

Dice Score for model1: 0.4422237885403691
Dice Score for model2: 0.9330127575408679
Dice Score for model3: 0.9401285453080809
Dice Score for model4: 0.9381434695526909
Dice Score for model5: 0.9394047113387476


In [32]:
dice = dice_score1(y_test, ensemble_mask.squeeze())

In [33]:
print("Dice Score for ensemble model :", dice)

Dice Score for ensemble model : 0.9392112573969069
